### New attempt to project

### Import libraries

In [1]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sqlite3 import Error
from sklearn.ensemble import RandomForestClassifier
import sqlite3
import pickle
import nltk
nltk.download('stopwords')
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/stevie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Read in data 

In [2]:
# Read in data frame.
# Add extra repeat rows to make training set larger 
df = pd.read_csv('ac_sheet5.csv')
df = pd.concat([df]*10000, ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   organizations  120000 non-null  object
 1   services       120000 non-null  object
dtypes: object(2)
memory usage: 1.8+ MB


### Text cleaning process

In [3]:
nltk.download('stopwords')
stemmer = PorterStemmer()
words = stopwords.words("english")
df['cleaned'] = df['services'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())




[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/stevie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Build vectorizer for words in dataset

In [4]:
vectorizer = TfidfVectorizer(min_df= 3, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))
final_features = vectorizer.fit_transform(df['cleaned']).toarray()
final_features.shape

(120000, 358)

### Create Pipeline thats transform data set to vector and place in model

In [6]:
#first we split our dataset into testing and training set:
# this block is to split the dataset into training and testing set 
X = df['cleaned']
Y = df['organizations']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)
# instead of doing these steps one at a time, we can use a pipeline to complete them all at once
pipeline = Pipeline([('vect', vectorizer), #Preprocessor
                     ('chi',  SelectKBest(chi2, k=300)), #Feature Selection
                     ('clf', RandomForestClassifier())]) #ML model type
# fitting our model and save it in a pickle for later use
model = pipeline.fit(X_train, y_train)
#with open('RandomForest.pickle', 'rb') as f:
#    pickle.dump()
ytest = np.array(y_test)
# confusion matrix and classification report(precision, recall, F1-score)
print(classification_report(ytest, model.predict(X_test)))
print(confusion_matrix(ytest, model.predict(X_test)))

              precision    recall  f1-score   support

      ACHSAM       1.00      1.00      1.00      2466
      ACHSID       1.00      1.00      1.00      2464
      ACHSPC       1.00      1.00      1.00      2533
        ACRM       1.00      1.00      1.00      2506
     GCAPFGP       1.00      1.00      1.00      2446
      GCAPHS       1.00      1.00      1.00      2486
     GCAPMIP       1.00      1.00      1.00      2523
   GCAPWICAC       1.00      1.00      1.00      2504
        SJAA       1.00      1.00      1.00      2528
      SJGEAT       1.00      1.00      1.00      2460
    VADVACSR       1.00      1.00      1.00      2513
     VADVNRG       1.00      1.00      1.00      2571

    accuracy                           1.00     30000
   macro avg       1.00      1.00      1.00     30000
weighted avg       1.00      1.00      1.00     30000

[[2466    0    0    0    0    0    0    0    0    0    0    0]
 [   0 2464    0    0    0    0    0    0    0    0    0    0]
 [   0 

In [ ]:
print(y_test)

### Test it out with random services

### Clean random service text before prediction

In [ ]:
new_df = pd.read_csv('ac_sheet6.csv')
new_df['cleaned'] = new_df['services'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

In [ ]:
new_df

In [119]:
model.predict(new_df['services'])

array(['GCAPMIP', 'VADVACSR', 'GCAPMIP', 'VADVACSR', 'GCAPMIP', 'GCAPMIP'],
      dtype=object)

In [122]:
df1 = pd.read_csv('ac_sheet.csv')

In [123]:
df1

,organizations,services
0,Volunteers of America Delaware Valley-Navigati...,"job training, educational placement , employme..."
1,Atlantic City Rescue Mission:,"Emergrency Shelter for men, women, and mothers..."
2,Gateway Community Action Partnership: WIC-Atla...,"Pregnant Postpartum Women, Infants, and Childr..."
3,Volunteers of America Delaware Valley: Atlanti...,linkage to mental health and substance abuse
4,Gateway Community Action Partnership: Foster G...,Intergenerational Program Senior Volunteer Opp...
5,AtlantiCare Health Services FQHC Adult Medicine,"We treat all patients, regardless of their ins..."
6,AtlantiCare Health Services FQHC Pediatric Clinic,substance abuse detox and management
7,AtlantiCare Health Services FQHC Infectious Di...,"mental health services, care for diabetes, ast..."
8,Gateway Community Action Partnership Head Star...,Early Childhood Education servicing Families w...
9,Gateway Community Action Partnership Male Init...,Fatherhood parenting support


In [7]:
txt = {'aids help'}

In [9]:
#txt = { "blair prepares to name poll date tony blair is likely to name 5 may as election day when parliament returns from its easter break  the bbc s political editor has learned.  andrew marr says mr blair will ask the queen on 4 or 5 april to dissolve parliament at the end of that week. mr blair has so far resisted calls for him to name the day but all parties have stepped up campaigning recently. downing street would not be drawn on the claim  saying election timing was a matter for the prime minister.  a number 10 spokeswoman would only say:  he will announce an election when he wants to announce an election.  the move will signal a frantic week at westminster as the government is likely to try to get key legislation through parliament. the government needs its finance bill  covering the budget plans  to be passed before the commons closes for business at the end of the session on 7 april.  but it will also seek to push through its serious and organised crime bill and id cards bill. mr marr said on wednesday s today programme:  there s almost nobody at a senior level inside the government or in parliament itself who doesn t expect the election to be called on 4 or 5 april.  as soon as the commons is back after the short easter recess  tony blair whips up to the palace  asks the queen to dissolve parliament ... and we re going.  the labour government officially has until june 2006 to hold general election  but in recent years governments have favoured four-year terms."}
txt = {'aids help'}
df1 = pd.DataFrame(txt)


In [10]:
model.predict(txt)

array(['GCAPMIP'], dtype=object)